In [1]:
import keras
#import keras.utils
from keras_preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
datagen = ImageDataGenerator(validation_split=0.2,
                             rescale=1./255 ,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True)
TRAIN_DIR = 'ahmraic'

train_generator = datagen.flow_from_directory(
    TRAIN_DIR, 
    subset='training',
        target_size=(200, 200),
        batch_size=32,
        class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    TRAIN_DIR,
    subset='validation',
        target_size=(200, 200),
        batch_size=32,
        class_mode='categorical'
)

Found 95 images belonging to 9 classes.
Found 18 images belonging to 9 classes.
